In [10]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_chroma import Chroma
from langchain_community.document_loaders import PyPDFLoader
from langchain.chains.question_answering import load_qa_chain

In [11]:
import os

In [ ]:
os.environ["GOOGLE_API_KEY"] = "api_key_here"

In [13]:
#Load dos modelos (Embedding e LLM)

embedding_model = GoogleGenerativeAIEmbeddings(model="models/text-embedding-004")
llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash", max_tokens=200)

In [14]:
#Carregar o PDF

pdf_link = "os-sertoes.pdf"

loader = PyPDFLoader(pdf_link, extract_images=False)
pages = loader.load_and_split()

In [15]:
#Separar em Chunks

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 4000,
    chunk_overlap = 20,
    length_function = len,
    add_start_index = True
)

chunks = text_splitter.split_documents(pages)

In [16]:
#Salvar no Vector DB - Chroma

db = Chroma.from_documents(chunks, embedding=embedding_model, persist_directory="text_index")

In [17]:
#Carregar o DB

vectordb = Chroma(persist_directory="text_index", embedding_function=embedding_model)

#Loading Retriever
retriever = vectordb.as_retriever(search_kwargs={"k":3})

#Construção da cadeia de prompt
chain = load_qa_chain(llm, chain_type="stuff")

In [23]:
def ask(question):
    context = retriever.get_relevant_documents(question)
    answer = chain({"input_documents": context, "question": question}, return_only_outputs=True)['output_text']
    return answer

In [ ]:
# Pergunta a serem respondidas sobre "Os Sertões" de Euclides da Cunha:
# 
# 1. Qual é a visão de Euclides da Cunha sobre o ambiente natural do sertão nordestino e como ele influencia a vida dos habitantes?
#2. Quais são as principais características da população sertaneja descritas por Euclides da Cunha? Como ele relaciona essas características com o ambiente em que vivem?
#3. Qual foi o contexto histórico e político que levou à Guerra de Canudos, segundo Euclides da Cunha?
#4. Como Euclides da Cunha descreve a figura de Antônio Conselheiro e seu papel na Guerra de Canudos?
#5. Quais são os principais aspectos da crítica social e política presentes em "Os Sertões"? Como esses aspectos refletem a visão do autor sobre o Brasil da época?

user_question = input("User: ")
answer = ask(user_question)
print("Answer: ", answer)